# Machine Translation

English-German Translation

# 1)- Importing key modules

In [0]:
#support both Python 2 and Python 3 with minimal overhead.
from __future__ import absolute_import, division, print_function
#ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [0]:
# What's life without style :). So, let's add style to our dataframes
#from IPython.core.display import HTML
#css = open('style-table.css').read() + open('style-notebook.css').read()
#HTML('<style>{}</style>'.format(css))

In [0]:
import pandas as pd 
import string 
import pickle
from pickle import dump
from pickle import load
from string import digits
import re 
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from numpy import array, argmax, random, take 
import matplotlib.pyplot as plt 
%matplotlib inline 
pd.set_option('display.max_colwidth', 200)

In [4]:
from keras.models import Model
from keras.models import Sequential 
from keras.layers import Dense, LSTM, Embedding,Input,RepeatVector
from keras.preprocessing.text import Tokenizer
from keras.callbacks import ModelCheckpoint 
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model 
from keras import optimizers 

Using TensorFlow backend.


In [5]:
!  pip install version_information

In [6]:
# first install: pip install version_information
%reload_ext version_information
%version_information pandas,re,sklearn, matplotlib,keras

Software versions
Python 3.6.8 64bit [GCC 8.0.1 20180414 (experimental) [trunk revision 259383]
IPython 5.5.0
OS Linux 4.14.137+ x86_64 with Ubuntu 18.04 bionic
pandas 0.24.2
re 2.2.1
sklearn 0.21.3
matplotlib 3.0.3
keras 2.2.5
Tue Sep 10 16:21:27 2019 UTC

# 2)- Reading Dataset

In [0]:
lines= pd.read_pickle('data_2009.pkl')

In [22]:
lines.shape

(2525, 2)

In [23]:
lines.head()

,eng,ger
0,Prague Stock Market falls to minus by the end of the trading day,Die Prager Börse stürzt gegen Geschäftsschluss ins Minus.
1,After a sharp drop in the morning,Nach dem steilen Abfall am Morgen konnte die Prager Börse die Verluste korrigieren.
2,Transactions with stocks from the Czech Energy Enterprise (ČEZ) reached nearly half of the regular daily trading.,Die Transaktionen mit den Aktien von ČEZ erreichten fast die Hälfte des normalen Tagesgeschäfts.
3,The Prague Stock Market immediately continued its fall from Monday at the beginning of Tuesday's trading,Die Prager Börse knüpfte gleich zu Beginn der Dienstagsgeschäfte an den Einbruch vom Montag an
4,This time the fall in stocks on Wall Street is responsible for the drop.,Diesmal lag der Grund für den Einbruch an der Wall Street.


As this is big data and I have a poor old computing machine. So, I ll use smaller sample. It got to be random to avoid sample biaseness

In [10]:
lines.sample(15)

,eng,ger
203,Regarding this,"Miroslav Čepický meint dazu: ""Für Verhandlungen mit potenziellen virtuellen Anbietern sind wir offen"
374,According to the prime minister,Nach Worten des Premiers wird es notwendig sein
1187,The collection of portraits of contemporary flamenco,Die Sammlung der Porträts des Flamencos von heute mit dem Titel `Flamenco-people' ist in der Galerie des El Fotómata ausgestellt
1105,On the face of it,Auf den ersten Blick hat sich nichts geändert.
748,For years,Seit Jahren sorgt ein mysteriöses Phänomen bei Forschen für Ratlosigkeit: Dunkle Materie beeinflusst die Geschwindigkeit von Raumsonden während so genannter Swing-by-Manöver.
2096,His Democratic colleague,Seine Kollegin aus Ohio
458,Should the pressure for more mergers increase,Sollte der Druck zu weiteren Fusionen steigen
809,"""This unpleasant saga ends with the victim taking her own life.",Diese unerfreuliche Geschichte endet mit dem Selbstmord des Opfers.
779,Right now we are checking their health,"Wir überprüfen gerade ihren Gesundheitszustand"""
1267,"""I wanted to remember her",`Ich wollte sie in Erinnerung zurückrufen


In [0]:
#lines = lines[:5000]

# 3)- Quick Preprocessing

In [0]:
# Lowercase all characters
lines.eng=lines.eng.apply(lambda x: x.lower())
lines.ger=lines.ger.apply(lambda x: x.lower())

In [0]:
# Remove quotes
lines.eng=lines.eng.apply(lambda x: re.sub("'", '', x))
lines.ger=lines.ger.apply(lambda x: re.sub("'", '', x))

In [0]:
exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
lines.eng=lines.eng.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines.ger=lines.ger.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [0]:
# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
lines.eng=lines.eng.apply(lambda x: x.translate(remove_digits))
lines.ger=lines.ger.apply(lambda x: x.translate(remove_digits))

In [0]:
# Remove extra spaces
lines.eng=lines.eng.apply(lambda x: x.strip())
lines.ger=lines.ger.apply(lambda x: x.strip())
lines.eng=lines.eng.apply(lambda x: re.sub(" +", " ", x))
lines.ger=lines.ger.apply(lambda x: re.sub(" +", " ", x))

In [0]:
# Add start and end tokens to target sequences. I am not German so , I am doing so for my ease
lines.ger = lines.ger.apply(lambda x : 'START_ '+ x + ' _END')

In [31]:
lines.ger[:5]

0                                           START_ die prager börse stürzt gegen geschäftsschluss ins minus _END
1                 START_ nach dem steilen abfall am morgen konnte die prager börse die verluste korrigieren _END
2    START_ die transaktionen mit den aktien von čez erreichten fast die hälfte des normalen tagesgeschäfts _END
3     START_ die prager börse knüpfte gleich zu beginn der dienstagsgeschäfte an den einbruch vom montag an _END
4                                          START_ diesmal lag der grund für den einbruch an der wall street _END
Name: ger, dtype: object

In [32]:
lines.sample(10)

,eng,ger
1342,what ballmer said this week confirms this idea,START_ was ballmer diese woche sagte _END
546,this may be causing some sleepless nights for longterm creditors in particular,START_ das dürfte vor allem langzeitgläubigern einige schlaflose nächte mit der entscheidung bereiten _END
664,he concluded his comments with a sideswipe my life has moved on,START_ und so schließen seinen einlassungen mit dem seitenhieb mein leben geht jedenfalls weiter _END
1230,according to a spokesman for the police,START_ laut einem polizeisprecher _END
1155,when he was freed glitter flew to bangkok where he was due to board a flight to london,START_ als er freigelassen wurde flog glitter nach bangkok _END
815,the man was on secondment from the ministry of defence when he left two highly classified documents on a train to waterloo,START_ der mann war vom verteidigungsministerium abgeordnet _END
2018,the reason for this is that the warsaw stock exchange is stateowned,START_ grund dafür ist _END
1286,he was an official enemy of richard nixon and a supporter of lost and won causes,START_ er war offizieller feind von richard nixon und erlitt wie verschiedene verluste sowohl auch gewinne von der stiftung _END
1019,paced by four kenyans on sunday,START_ begleitet von vier kenianern am sonntag hatte gebrselassie den mittelpunkt der strecke in erreicht _END
916,for instance,START_ zum beispiel _END


In [0]:
# Vocabulary of English
all_eng_words=set()
for eng in lines.eng:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)
# Vocabulary of German 
all_german_words=set()
for ger in lines.ger:
    for word in ger.split():
        if word not in all_german_words:
            all_german_words.add(word)

In [34]:
# Max Length of source sequence
import numpy as np
lenght_list=[]
for l in lines.eng:
    lenght_list.append(len(l.split(' ')))
max_length_src = np.max(lenght_list)
max_length_src

70

In [35]:
# Max Length of target sequence
lenght_list=[]
for l in lines.ger:
    lenght_list.append(len(l.split(' ')))
max_length_tar = np.max(lenght_list)
max_length_tar

52

# Make a threshold here

### 3a)- Defining input and target

In [36]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_german_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_german_words)
num_encoder_tokens, num_decoder_tokens

(5454, 6522)

In [37]:
num_decoder_tokens += 1 # For zero padding
num_decoder_tokens

6523

In [0]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [0]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [41]:
print(reverse_input_char_index)

{1: 'a', 2: 'aac', 3: 'abandon', 4: 'abc', 5: 'abducted', 6: 'abductors', 7: 'abetting', 8: 'ability', 9: 'able', 10: 'about', 11: 'above', 12: 'abovementioned', 13: 'abroad', 14: 'absenteeism', 15: 'absolute', 16: 'absolutely', 17: 'abuse', 18: 'academia', 19: 'accelerate', 20: 'acceleration', 21: 'accelerator', 22: 'accentuating', 23: 'accept', 24: 'acceptable', 25: 'accepted', 26: 'access', 27: 'accident', 28: 'accidental', 29: 'accidentally', 30: 'accidents', 31: 'accommodatingly', 32: 'accommodation', 33: 'accompanied', 34: 'accordance', 35: 'according', 36: 'accordingly', 37: 'accounting', 38: 'accounts', 39: 'accrue', 40: 'accumulators', 41: 'accusation', 42: 'accusations', 43: 'accuse', 44: 'accused', 45: 'accusing', 46: 'accustomed', 47: 'achieve', 48: 'achieved', 49: 'achievement', 50: 'acquired', 51: 'acquisition', 52: 'across', 53: 'act', 54: 'action', 55: 'actions', 56: 'active', 57: 'activities', 58: 'activity', 59: 'actors', 60: 'actress', 61: 'actually', 62: 'adam', 63:

In [42]:
print(reverse_target_char_index)

{1: 'START_', 2: '_END', 3: 'a', 4: 'aac', 5: 'ab', 6: 'abbringen', 7: 'abc', 8: 'abend', 9: 'aber', 10: 'abfall', 11: 'abgabenordnung', 12: 'abgebildet', 13: 'abgebogen', 14: 'abgegrenzt', 15: 'abgelehnt', 16: 'abgeltungssteuerzahlungen', 17: 'abgeordnet', 18: 'abgeordnete', 19: 'abgeordneten', 20: 'abgeordnetenhaus', 21: 'abgeordnetenkammer', 22: 'abgeordneter', 23: 'abgeschlossen', 24: 'abgesehen', 25: 'abgestimmt', 26: 'abgewiesen', 27: 'abhinge', 28: 'abhängig', 29: 'abhängiger', 30: 'abhöraktion', 31: 'abkauf', 32: 'abkommen', 33: 'abkommens', 34: 'ablauf', 35: 'ablaufen', 36: 'ablehnung', 37: 'ableiten', 38: 'abreise', 39: 'absagen', 40: 'absatzmarkt', 41: 'absatzmärkte', 42: 'absatzmärkten', 43: 'abschaffung', 44: 'abschlussurteil', 45: 'abschreibungen', 46: 'abschusszahlen', 47: 'abseits', 48: 'absenzen', 49: 'absicht', 50: 'absichtlich', 51: 'absolut', 52: 'absolute', 53: 'abspielen', 54: 'abstand', 55: 'abstimmen', 56: 'abstimmung', 57: 'abstimmungsniederlage', 58: 'abstufun

### 3b)-Train - Test Split

For validation

In [43]:
X, y = lines.eng, lines.ger #X being input, y being target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
X_train.shape, X_test.shape

((2020,), (505,))

**Save the train and test dataframes for reproducing the results later, as they are shuffled**

In [0]:
X_train.to_pickle('X_train.pkl')
X_test.to_pickle('X_test.pkl')

In [0]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

# 4)-Encoder - Decoder Model Architecture

In [0]:
latent_dim = 50

In [47]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [49]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [0]:
#from IPython.display import Image
#Image(retina=True, filename='train.png')

In [0]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 15

In [52]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 50)     272700      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 50)     326150      input_2[0][0]                    
____________________________________________________________________________________________

In [53]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)


Epoch 1/15
15/15 [==============================] - 8s 514ms/step - loss: 8.6017 - acc: 0.0909 - val_loss: 8.0899 - val_acc: 0.1013
Epoch 2/15
15/15 [==============================] - 5s 330ms/step - loss: 7.5900 - acc: 0.1023 - val_loss: 7.3998 - val_acc: 0.1044
Epoch 3/15
15/15 [==============================] - 5s 330ms/step - loss: 7.0569 - acc: 0.1028 - val_loss: 7.2367 - val_acc: 0.0988
Epoch 4/15
15/15 [==============================] - 5s 327ms/step - loss: 6.8835 - acc: 0.1025 - val_loss: 7.1771 - val_acc: 0.1041
Epoch 5/15
15/15 [==============================] - 5s 326ms/step - loss: 6.8052 - acc: 0.1024 - val_loss: 7.2367 - val_acc: 0.1013
Epoch 6/15
15/15 [==============================] - 5s 329ms/step - loss: 6.7858 - acc: 0.1026 - val_loss: 7.2965 - val_acc: 0.1044
Epoch 7/15
15/15 [==============================] - 5s 324ms/step - loss: 6.7697 - acc: 0.1031 - val_loss: 7.3804 - val_acc: 0.0988
Epoch 8/15
15/15 [==============================] - 5s 325ms/step - loss: 6

In [0]:
# save model
model.save_weights('translate.h5')

In [0]:
model.load_weights('translate.h5')

# Inference Setup

In [0]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

# Decode sample sequeces

In [0]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    
    
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']
  

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

# Evaluation on Train Dataset

In [0]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [59]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual German Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted German Translation:', decoded_sentence[:-4])

Input English sentence: after the waves
Actual German Translation:  erst die wellen 
Predicted German Translation:  


In [60]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual German Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted German Translation:', decoded_sentence[:-4])

Input English sentence: soon all is quiet in the bus
Actual German Translation:  schnell wird es ruhig im bus 
Predicted German Translation:  


In [61]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual German Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted German Translation:', decoded_sentence[:-4])

Input English sentence: when you ask taxpayers to take extraordinary measures because of the irresponsibility of some
Actual German Translation:  wenn man die steuerzahler darum bittet 
Predicted German Translation:  


# Evaluation on Validation Dataset

In [0]:
val_gen = generate_batch(X_test, y_test, batch_size = 1)
k=-1

In [63]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual German Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted German Translation:', decoded_sentence[:-4])

Input English sentence: aig is ready to explore the sale of most of its activities beyond its international life insurance and its pension business in the united states
Actual German Translation:  aig ist dazu bereit 
Predicted German Translation:  


In [64]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual German Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted German Translation:', decoded_sentence[:-4])

Input English sentence: in a rare shared opinion with a statement by o
Actual German Translation:  die pressesprecherin von tmobile martina kemrová fügt in seltener übereinstimmung mit der erklärung von o hinzu bislang haben wir noch kein lebensfähiges und beidseitig vorteilhaftes projekt gefunden 
Predicted German Translation:  


In [65]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual German Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted German Translation:', decoded_sentence[:-4])

Input English sentence: the name and tax number of our company were placed incorrectly on the stamp
Actual German Translation:  unser firmenname und die steuernummer waren auf dem stempel falsch angebracht 
Predicted German Translation:  
